# Lab. 1-2 Text2SQL Advanced (Athena & Amazon S3)

#### 이 실습에서는 Text2SQL을 활용해서 S3에 저장된 데이터에 Athena 쿼리로 접근하는 방법을 실습합니다. (아키텍처는 1.basic-athena.ipynb 와 동일합니다)

#### 많은 Text2SQL 시나리오가 사용자의 복잡한 요청과 쿼리 작성패턴으로 이루어지기 때문에, LLM 호출을 여러번의 작업으로 분리하는 최적화 방법들이 시도됩니다.

#### 이 노트북에서는 이론 과정에 소개된 논문 중 **DIN-SQL**의 구현 방식을 테스트해보겠습니다.
![Intro](../images/text2sql/athena-s3-2.png)

1. Schema Linking - 쿼리에 사용할 스키마를 연결합니다.
2. Classification & Decomposition - 쿼리의 난이도를 분류하고, 요청된 내용이 중첩된 쿼리를 포함한다면 이를 여러 명령으로 분리시킵니다.
3. SQL Generation - 분류된 난이도에 맞는 쿼리 작성 프롬프트를 호출합니다. 예를 들어, 난이도가 Medium 이상이라면 CoT(Chain-of-Thouhgt) 프롬프팅을 활용합니다.
4. Self Correction - 쿼리 생성 결과를 검증합니다.

#### 다양한 Text2SQL 구현 패턴이 있지만, 큰 틀에서 위의 흐름을 따르는 경우가 많습니다.

*이 노트북의 실습은 [원문 워크샵]((https://github.com/aws-samples/text-to-sql-bedrock-workshop)) 시나리오를 참조하여 구성되었습니다. 

## Step 0: 라이브러리 설치 및 Athena 연결

In [ ]:
!python -m ensurepip --upgrade
!pip install "sqlalchemy" --quiet
!pip install "boto3>=1.34.116"  --quiet
!pip install "jinja2" --quiet
!pip install "botocore" --quiet
!pip install "pandas" --quiet
!pip install "PyAthena" --quiet
!pip install "faiss-cpu" --quiet

In [1]:
import boto3
import sys

sys.path.append('../')
from libs.din_sql import din_sql_lib as dsl

In [ ]:
ATHENA_CATALOG_NAME = '' # check https://us-west-2.console.aws.amazon.com/cloudformation/home?region=us-west-2#/stacks
ATHENA_RESULTS_S3_LOCATION = '' # check https://us-west-2.console.aws.amazon.com/cloudformation/home?region=us-west-2#/stacks
DB_NAME = "tpcds1"

In [10]:
!aws athena list-data-catalogs

{
    "DataCatalogsSummary": [
        {
            "CatalogName": "AwsDataCatalog",
            "Type": "GLUE"
        }
    ]
}


In [12]:
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)

In [13]:
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

attempting to connect to athena database with connection string: awsathena+rest://:@athena.us-west-2.amazonaws.com:443/text2sql?s3_staging_dir=s3://s3://text2sql-db/results/&catalog_name=AwsDataCatalog
connected to database successfully.


## Step 1: Schema Linking 모듈

### 사용할 DB의 스키마 정보 확보

In [14]:
return_sql= din_sql.find_fields(db_name=DB_NAME)
print(return_sql)

database name specified and found, inspecting only 'text2sql'
Table album, columns = [albumid,title,artistid]
Table artist, columns = [albumid,title,artistid]
Table customer, columns = [albumid,title,artistid]
Table employee, columns = [albumid,title,artistid]
Table genre, columns = [albumid,title,artistid]
Table invoice, columns = [albumid,title,artistid]
Table invoiceline, columns = [albumid,title,artistid]
Table mediatype, columns = [albumid,title,artistid]
Table playlist, columns = [albumid,title,artistid]
Table playlisttrack, columns = [albumid,title,artistid]
Table track, columns = [albumid,title,artistid]



### 스키마 선별을 위한 프롬프트 구성 & LLM 요청

In [15]:
question = "Which customer spent the most money in the web store?"

# 프롬프트 구성 : Instruction + Few shot samples + Question
schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)

# Word-in-mouth 추가 : Instruction for CoT
word_in_mouth_schema_link = f'A. Let’s think step by step. In the question "{question}", we are asked:'

# LLM 질문 & 답변 생성
schema_links = din_sql.llm_generation(
                    schema_links_prompt,
                    stop_sequences=['</links>'],
                    word_in_mouth=word_in_mouth_schema_link
                    )

database name specified and found, inspecting only 'text2sql'
Successfully invoked model anthropic.claude-3-sonnet-20240229-v1:0


### 결과 확인

In [ ]:
print(f"{word_in_mouth_schema_link}{schema_links}")

# 참고 : 위에서 stop_sequences로 </links>를 걸었기 때문에, 스키마 링크가 출력되는 직후 답변이 중단됨

In [ ]:
links = schema_links.split('<links>')[1].replace('\n','')
links

#### 이제 자연어 질문 처리에 필요한 스키마 링크가 확보되었습니다.

## Step 2: Classification 모듈

### 쿼리 난이도 분류를 위한 프롬프트 구성 & LLM 요청

In [ ]:
# 프롬프트 구성 : Instruction + Few shot samples + Link + Question
classification_prompt = din_sql.classification_prompt_maker(question, DB_NAME, links)

# Word-in-mouth 추가 : Instruction for CoT
word_in_mouth_classification = "A: Let’s think step by step."

# LLM 질문 & 답변 생성
classification = din_sql.llm_generation(
                    classification_prompt,
                    stop_sequences=['</label>'],
                    word_in_mouth=word_in_mouth_classification
                    )

### 결과 확인

In [ ]:
print(f"{word_in_mouth_classification}{classification}")

In [ ]:
predicted_class = classification.split('<label>')[1].replace('\n','')
predicted_class

## Step 3: SQL Generation 모듈

### 쿼리 생성을 위한 프롬프트 구성 & LLM 요청

In [ ]:
# 프롬프트 구성 : 쿼리의 예상 난이도(EASY / NON-NESTED / NESTED)에 따라 다른 프롬프트를 구성
sql_tag_start = '```sql'
sql_generation_prompt = din_sql.medium_prompt_maker(
                        test_sample_text=question, 
                        database=DB_NAME, 
                        schema_links=links,
                        sql_tag_start=sql_tag_start,
                        sql_tag_end='```')

# Word-in-mouth 추가 : split을 위한 태그 삽입 
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
#word_in_mouth_medium_prompt = "A: Let’s think step by step. For creating the SQL for the given question, we need to join tables. First, create an intermediate representation, then use it to construct the SQL query.\n Intermediate_representation:"

# LLM 질문 & 답변 생성
sql_qry = din_sql.llm_generation(
                        prompt=sql_generation_prompt,
                        stop_sequences=['</example>'],
                        word_in_mouth=word_in_mouth_medium_prompt
                    )

In [ ]:
print(f"{word_in_mouth_medium_prompt}{sql_qry}")

In [ ]:
SQL = sql_qry.split('```')[0].strip()
#SQL = sql_qry.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

### 생성된 SQL 쿼리 테스트

In [ ]:
import pandas as pd
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

### 결과 검증

In [ ]:
validation_query = """
    SELECT "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    , SUM("ws"."ws_net_paid") as total_sales
    FROM "customer" "c" 
    JOIN "web_sales" "ws" 
        ON "ws"."ws_bill_customer_sk" = "c"."c_customer_sk"   
    GROUP BY "c"."c_customer_sk"
    , "c"."c_first_name"
    , "c"."c_last_name"
    ORDER BY total_sales desc
    limit 10
"""
validation_set = din_sql.query(validation_query)
pd.DataFrame(validation_set)

## Step 4: Self Correction 모듈

위에서는 쿼리를 먼저 실행했지만, 원래 쿼리를 실행하기 전에 Self Correction을 거치는 것이 일반적 접근 방법입니다.

여기에서는 하나의 LLM으로 간단히 처리했지만, 필요에 따라 Multi-LLM을 활용에 교차 검증하거나, 쿼리 플랜을 검토하기도 합니다.

### 쿼리 정합성 검증

In [ ]:
revised_sql = din_sql.debugger_generation(
            prompt=din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
            ).replace("\n", " ")
print(f"{revised_sql}")

In [ ]:
SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
print(f"{SQL}")

In [ ]:
result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

## End-to-End 수행

전체 과정을 한 번에 실행한다면, LLM 호출이 연쇄적으로 발생하며 다소 긴 시간이 소요될 수도 있습니다.

(여기에서는 Athena 조회 성능의 특성으로 인해 DB에 비해 보다 오랜시간이 소요됩니다.)

In [ ]:
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

din_sql = dsl.DIN_SQL(bedrock_model_id=model_id)
din_sql.athena_connect(catalog_name=ATHENA_CATALOG_NAME, 
               db_name=DB_NAME, 
               s3_prefix=ATHENA_RESULTS_S3_LOCATION)

In [ ]:
question = "카탈로그가 가장 많이 판매된 해가 언제인가요?"

# Schema Linking 모듈
print("Schema Linking")
schema_links_prompt = din_sql.schema_linking_prompt_maker(question, DB_NAME)
word_in_mouth_schema_link = f'A. Let’s think step by step. In the question "{question}", we are asked:'
%time schema_links = din_sql.llm_generation(schema_links_prompt, stop_sequences=['</links>'], word_in_mouth=word_in_mouth_schema_link)
links = schema_links.split('<links>')[1].replace('\n','')

# Classification 모듈
print("\nClassification and Decomposition")
classification_prompt = din_sql.classification_prompt_maker(question, DB_NAME, links)
word_in_mouth_classification = "A: Let’s think step by step."
%time classification = din_sql.llm_generation(classification_prompt, stop_sequences=['</label>'], word_in_mouth=word_in_mouth_classification)
predicted_class = classification.split('<label>')[1].replace('\n','')

# SQL Generation 모듈
print("\nSQL Generation")
sql_tag_start='```sql'
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
sql_generation_prompt = din_sql.medium_prompt_maker(test_sample_text=question, database=DB_NAME, schema_links=links, sql_tag_start=sql_tag_start, sql_tag_end='```')
word_in_mouth_medium_prompt = f"SQL: {sql_tag_start}"
%time sql_qry = din_sql.llm_generation(prompt=sql_generation_prompt, stop_sequences=['</example>'], word_in_mouth=word_in_mouth_medium_prompt)
SQL = sql_qry.split('```')[0].strip()

# Self Correction 모듈
print("\nSelf Correction") 
debug_prompt = din_sql.debugger(question, DB_NAME, SQL, sql_dialect='presto')
%time revised_sql = din_sql.debugger_generation(prompt=debug_prompt).replace("\n", " ")

# 쿼리 실행
print("\nQuery Execution") 
#SQL = revised_sql.split('```sql')[1].split('```')[0].strip()
SQL = revised_sql.split('```')[0].strip()
%time result_set = din_sql.query(SQL)
pd.DataFrame(result_set)

### 여기에서는 Text2SQL 과정을 여러 LLM의 호출로 분리하여 처리하는 방법을 사용했습니다.
### 이 방법은 직관적이지만 예외 발생 등에 대한 유연성이 부족합니다. 이어서는 LLM 호출 프레임워크를 활용한 구성 방법을 알아보겠습니다.